## Imports

In [1]:
import tensorflow as tf
# from keras.layers.normalization import layer_normalization

In [2]:
from tensorflow import keras
from keras.layers.normalization import layer_normalization
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
import warnings
import numpy as np
# import matplotlib.pyplot as plt
import cv2
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint, EarlyStopping
import sklearn as sk
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
warnings.simplefilter(action='ignore', category=FutureWarning)

## Train and Test image directory paths

In [3]:
train_path = r'gestures\train'
test_path = r'gestures\test'

In [4]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=train_path, target_size=(64,64), class_mode='categorical', batch_size=10, shuffle=True)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=test_path, target_size=(64,64), class_mode='categorical', batch_size=10, shuffle=True)

Found 755 images belonging to 5 classes.
Found 755 images belonging to 5 classes.


## Building the model

In [5]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(64,64,3)))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = 'same'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding = 'valid'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Flatten())

model.add(Dense(64, activation ="relu"))
model.add(Dense(128, activation ="relu"))
model.add(Dense(128, activation ="relu"))
model.add(Dense(5, activation ="softmax")) # Change the number of classes to the number of gestures added

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=0.0001)
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')

model.compile(optimizer=SGD(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=0.0005)
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')

history2 = model.fit(train_batches, epochs=100, callbacks=[reduce_lr, early_stop],  validation_data = test_batches)

Epoch 1/100
76/76 [==============================] - 6s 55ms/step - loss: 2.7840 - accuracy: 0.9192 - val_loss: 0.2156 - val_accuracy: 0.9470
Epoch 2/100
76/76 [==============================] - 4s 51ms/step - loss: 0.0050 - accuracy: 1.0000 - val_loss: 0.2096 - val_accuracy: 0.9510
Epoch 3/100
76/76 [==============================] - 4s 55ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.1977 - val_accuracy: 0.9536
Epoch 4/100
76/76 [==============================] - 4s 46ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.1998 - val_accuracy: 0.9550
Epoch 5/100
76/76 [==============================] - 3s 45ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.1995 - val_accuracy: 0.9563


In [ ]:
history2.history

## Saving our model

In [6]:
model.save('signForBW') # For Ashwith's system
# model.save('sign_model.h5') # For Manasi's system

INFO:tensorflow:Assets written to: signForColor\assets


## Metrics

In [8]:
imgs, labels = next(test_batches) 
scores = model.evaluate(imgs, labels, verbose=0)
print('{0} of {1}; {2} of {3}%'
.format(model.metrics_names[0], scores[0], model.metrics_names[1], scores[1]*100))
scores

loss of 0.39209672808647156; accuracy of 89.99999761581421%


[0.39209672808647156, 0.8999999761581421]

In [9]:
imgs, labels = next(test_batches)
model = keras.models.load_model("signForBW") # Change the model name accordingly
# new_model = tf.keras.models.load_model('saved_model/my_model')
scores = model.evaluate(imgs, labels, verbose=0)
print('{0} of {1}; {2} of {3}%'
.format(model.metrics_names[0], scores[0], model.metrics_names[1], scores[1]*100))

loss of 0.08407296240329742; accuracy of 100.0%


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 31, 31, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0

## Predictions

In [11]:
imgs, labels = next(test_batches) 

word_dict = {0:'One', 1:'Two', 2:'Three', 3:'I Love You', 4:'Little'} # Sort it according to the folder names

predictions = model.predict(imgs, verbose=0)
predicted_labels = []
actual_labels = []

print("Predicted labels")
for ind, i in enumerate(predictions):
    print(word_dict[np.argmax(i)])
    predicted_labels.append(word_dict[np.argmax(i)])
    
print('Actual labels')
for i in labels:
    print(word_dict[np.argmax(i)])
    actual_labels.append(word_dict[np.argmax(i)])


Predicted labels
One
A
C
D
A
D
I Love You
Two
E
One
Actual labels
One
A
C
D
A
D
I Love You
Two
E
One


In [58]:
print ("Precision", sk.metrics.precision_score(actual_labels, predicted_labels, average='macro'))
print ("Recall", sk.metrics.recall_score(actual_labels, predicted_labels, average='macro'))
print ("f1_score", sk.metrics.f1_score(actual_labels, predicted_labels, average='macro'))
print ("confusion_matrix: ")
print (sk.metrics.confusion_matrix(actual_labels, predicted_labels))

Precision 0.9333333333333332
Recall 0.9333333333333333
f1_score 0.9199999999999999
confusion_matrix: 
[[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 3 0 0]
 [0 0 0 2 0]
 [0 0 0 1 2]]


In [39]:
labels

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], dtype=float32)

In [40]:
predictions

array([[9.8876858e-01, 1.0727884e-02, 5.0357770e-04],
       [2.1651026e-03, 9.9640560e-01, 1.4292375e-03],
       [9.9556869e-01, 3.3518095e-03, 1.0795402e-03],
       [5.9244330e-06, 1.7637222e-04, 9.9981779e-01],
       [3.8885388e-05, 2.0312744e-03, 9.9792981e-01],
       [4.3068114e-03, 9.9475753e-01, 9.3558541e-04],
       [2.5023993e-03, 7.7351741e-03, 9.8976249e-01],
       [9.9923503e-01, 4.6021870e-04, 3.0467054e-04],
       [1.8225446e-02, 4.9001750e-01, 4.9175704e-01],
       [9.7995055e-01, 1.9079464e-02, 9.6999458e-04]], dtype=float32)